In [9]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

#Get the stock quote
df = web.DataReader('TSLA', data_source='yahoo', start='2012-01-01', end='2019-12-17')

#Show the data
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2012-01-03,5.900000,5.530000,5.788000,5.616000,4640500,5.616000
2012-01-04,5.734000,5.500000,5.642000,5.542000,3150500,5.542000
2012-01-05,5.586000,5.370000,5.552000,5.424000,5027500,5.424000
2012-01-06,5.558000,5.282000,5.440000,5.382000,4931500,5.382000
2012-01-09,5.498000,5.224000,5.400000,5.450000,4485000,5.450000
...,...,...,...,...,...,...
2019-12-11,71.438004,70.218002,70.375999,70.540001,34489000,70.540001
2019-12-12,72.547997,70.646004,70.984001,71.935997,38819500,71.935997
2019-12-13,73.042000,70.928001,72.209999,71.678001,32854500,71.678001


In [10]:
from datetime import datetime
import re
import json
import csv
from io import StringIO
import csvfile as csvfile
from bs4 import BeautifulSoup
import requests

url_stats = 'https://uk.finance.yahoo.com/quote/{}/key-statistics?p={}'
companies = ['TSLA', 'MSFT', 'AAPL', 'AMZN', 'FB', 'GOOG', 'NFLX', 'SPOT', 'ZM', 'DBX', 'KO', 'DIS', 'PEP', 'ADBE', 'SONO', 'AMD', 'NVDA', 'INTC', 'NKE', 'NOK']
stock = 'TSLA'

def datetime_to_float(d):
    return d.timestamp()

#Statistics
response = requests.get(url_stats.format(stock, stock))
soup = BeautifulSoup(response.text, 'html.parser')
pattern = re.compile(r'\s--\sData\s--\s')
script_data = soup.find('script', text=pattern).contents[0]
start = script_data.find("context")-2
json_data = json.loads(script_data[start:-12])

now = datetime.today().strftime('%Y-%m-%d')
# print(now)

#Historical Stock Data
stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?'

params = {
    'start' : now,
    'interval' : '1d',
    'range' : '1y',
    'events' : 'history'
}

# response = requests.get(stock_url.format(stock), params=params)
# response.text

for s in companies:
    response = requests.get(stock_url.format(s), params=params)
    response.text

    file = StringIO(response.text)
    reader = csv.DictReader(file)
    with open('./csvs/'+s+'.csv', 'w', newline='') as f:
        fieldnames = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
        writer = csv.DictWriter(f, fieldnames=fieldnames)

        #writer.writeheader() #Not needed
        for row in reader:
            # writer.writerow({'Date': datetime_to_float(datetime.strptime(row['Date'], "%Y-%m-%d")), 'Open': row['Open'], 'High': row['High'], 'Low': row['Low'], 'Close': row['Close'], 'Adj Close': row['Adj Close'], 'Volume': row['Volume']})
            writer.writerow({'Date': row['Date'], 'Open': row['Open'], 'High': row['High'], 'Low': row['Low'], 'Close': row['Close'], 'Adj Close': row['Adj Close'], 'Volume': row['Volume']})

# string_date = "2020-01-10"
# datetimeObj = datetime.strptime(row['Date'], "%Y-%m-%d")
# print(datetimeObj.date())
# print(datetime_to_float(datetimeObj))